<a href="https://colab.research.google.com/github/MarioROT/IHLT-MAI/blob/main/Session7_MarioRosas_AlamLopez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab session 7 (Word Sequences) - IHLT

**Students:**
- Mario Rosas
- Alam Lopez

**Lab Professor:** Salvador Medina Herrera

## Paraphrases Template

In [ ]:
%%shell
git clone https://github.com/mariorot/IHLT-MAI.git
cd 'IHLT-MAI'
mv 'complementary_material' /content/
mv scripts /content/

Cloning into 'IHLT-MAI'...
remote: Enumerating objects: 269, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 269 (delta 53), reused 30 (delta 21), pack-reused 190
Receiving objects: 100% (269/269), 282.97 KiB | 2.64 MiB/s, done.
Resolving deltas: 100% (143/143), done.


In [ ]:
from scripts.compute_metrics import ComputeMetrics
from scripts.text_preprocessing import TextPreprocessing
import pandas as pd
pd.set_option("display.precision", 4)
from scipy.stats import pearsonr
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus.reader.wordnet import VERB, NOUN, ADJ, ADV

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
dt = pd.read_csv('complementary_material/test-gold/STS.input.SMTeuroparl.txt',sep='\t',header=None)
dt['gs'] = pd.read_csv('complementary_material/test-gold/STS.gs.SMTeuroparl.txt',sep='\t',header=None)

## TODO
1. Read all pairs of sentences of the SMTeuroparl files of test set within the evaluation framework of the project. Compute their similarities by considering *lemmas* and *Jaccard distance*. <br>

2. Compare the results with those in session 2 (document structure) in which words were considered. : <br>

3. Compare the results with gold standard by giving the pearson correlation between them. <br>

4. Questions (justify the answers): <br>

      Which is better: words or lemmas?

      Do you think that could perform better for any pair of texts?

In [ ]:
pos_map = {'n': ['NN', 'NNS', 'NNP', 'NNPS'],
           'v': ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'],
           'j': ['JJ', 'JJR', 'JJS'],
           'r': ['RB', 'RBR', 'RBS']}
correction = {'N': NOUN,
              'V':VERB,
              'J':ADJ,
              'R':ADV}

In [ ]:
tp = TextPreprocessing()

# -- Data cleaning
dt[2]=tp.clean_data(list(dt[0]))
dt[3]=tp.clean_data(list(dt[1]))

# -- Tokenization
# NLTK
dt[4] = tp.tokenize_data(list(dt[0]),'nltk')
dt[5] = tp.tokenize_data(list(dt[1]),'nltk')
# spaCy
dt[6] = tp.tokenize_data(list(dt[0]),'spacy')
dt[7] = tp.tokenize_data(list(dt[1]),'spacy')

# -- Lemmatization
# NLTK
dt[8]=tp.lemmatize_data(list(dt[0]),'nltk',False)
dt[9]=tp.lemmatize_data(list(dt[1]),'nltk',False)
# spaCy
dt[10]=tp.lemmatize_data(list(dt[0]),'spacy')
dt[11]=tp.lemmatize_data(list(dt[1]),'spacy')

# -- Lemmatization with cleaned data
# NLTK
dt[12]=tp.lemmatize_data(list(dt[2]),'nltk')
dt[13]=tp.lemmatize_data(list(dt[3]),'nltk')
# spaCy
dt[14]=tp.lemmatize_data(list(dt[2]),'spacy')
dt[15]=tp.lemmatize_data(list(dt[3]),'spacy')

#--Word Desambiguation with cleaned Lemmas NLTK

dt[16]= tp.lesk_lemmatize_data(list(dt[12]),'nltk')
dt[17]= tp.lesk_lemmatize_data(list(dt[13]),'nltk')


#--Word Desambiguation with cleaned Lemmas Spacy

dt[18]= tp.lesk_lemmatize_data(list(dt[14]),'nltk')
dt[19]= tp.lesk_lemmatize_data(list(dt[15]),'nltk')


#--Word Desambiguation with Lemmas NLTK

dt[20]=  tp.lesk_lemmatize_data(list(dt[8]),'nltk')
dt[21]= tp.lesk_lemmatize_data(list(dt[9]),'nltk')

#--Word Desambiguation with Lemmas Spacy

dt[22]= tp.lesk_lemmatize_data(list(dt[10]),'nltk')
dt[23]= tp.lesk_lemmatize_data(list(dt[11]),'nltk')

#--Word Desambiguation cleaned data

dt[24]= tp.lesk_lemmatize_data(list(dt[2]),'nltk')
dt[25]= tp.lesk_lemmatize_data(list(dt[3]),'nltk')



The code for the tokenization is at: https://github.com/MarioROT/IHLT-MAI/blob/main/scripts/text_preprocessing.py

The code for jaccard distance computation is at: https://github.com/MarioROT/IHLT-MAI/blob/main/scripts/compute_metrics.py

In [ ]:
# -- Jaccard computation
# Tokens
cm_clean_nltk = ComputeMetrics(dt[[2,3]].to_numpy(), ['jaccard'], 1)
dt['jaccard_clean'] = cm_clean_nltk.do()['jaccard']

cm_nltk = ComputeMetrics(dt[[4,5]].to_numpy(), ['jaccard'], 1)
dt['jaccard_token_nltk'] = cm_nltk.do()['jaccard']

cm_spacy = ComputeMetrics(dt[[6,7]].to_numpy(), ['jaccard'], 1)
dt['jaccard_token_spacy'] = cm_spacy.do()['jaccard']

# Lemmas
cm_lemma_nltk = ComputeMetrics(dt[[8,9]].to_numpy(), ['jaccard'], 1)
dt['jaccard_lemma_nltk']=cm_lemma_nltk.do()['jaccard']

cm_lemma_spacy = ComputeMetrics(dt[[10,11]].to_numpy(), ['jaccard'], 1)
dt['jaccard_lemma_spacy']=cm_lemma_spacy.do()['jaccard']

cm_lemma_clean_nltk = ComputeMetrics(dt[[12,13]].to_numpy(), ['jaccard'], 1)
dt['jaccard_lemma_clean_nltk']=cm_lemma_clean_nltk.do()['jaccard']

cm_lemma_clean_spacy = ComputeMetrics(dt[[14,15]].to_numpy(), ['jaccard'], 1)
dt['jaccard_lemma_clean_spacy']=cm_lemma_clean_spacy.do()['jaccard']

#WSD

cm_wsd_lemma_clean_nltk = ComputeMetrics(dt[[16,17]].to_numpy(), ['jaccard'], 1)
dt['jaccard_lemma_wsd_clean_nltk']=cm_wsd_lemma_clean_nltk.do()['jaccard']

cm_wsd_lemma_clean_spacy = ComputeMetrics(dt[[18,19]].to_numpy(), ['jaccard'], 1)
dt['jaccard_lemma_wsd_clean_spacy']=cm_wsd_lemma_clean_spacy.do()['jaccard']

cm_wsd_lemma_nltk = ComputeMetrics(dt[[20,21]].to_numpy(), ['jaccard'], 1)
dt['jaccard_lemma_wsd_nltk']=cm_wsd_lemma_clean_nltk.do()['jaccard']

cm_wsd_lemma_spacy = ComputeMetrics(dt[[22,23]].to_numpy(), ['jaccard'], 1)
dt['jaccard_lemma_wsd_spacy']=cm_wsd_lemma_clean_spacy.do()['jaccard']

cm_wsd_cleaned_data = ComputeMetrics(dt[[24,25]].to_numpy(), ['jaccard'], 1)
dt['jaccard_wsd_cleaned']=cm_wsd_cleaned_data.do()['jaccard']


Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...


In [ ]:
dt[['gs','jaccard_token_nltk','jaccard_token_spacy','jaccard_clean','jaccard_lemma_nltk','jaccard_lemma_clean_nltk','jaccard_lemma_spacy','jaccard_lemma_clean_spacy','jaccard_lemma_wsd_clean_nltk','jaccard_lemma_wsd_nltk','jaccard_lemma_wsd_clean_spacy','jaccard_lemma_wsd_spacy','jaccard_wsd_cleaned']]

,gs,jaccard_token_nltk,jaccard_token_spacy,jaccard_clean,jaccard_lemma_nltk,jaccard_lemma_clean_nltk,jaccard_lemma_spacy,jaccard_lemma_clean_spacy,jaccard_lemma_wsd_clean_nltk,jaccard_lemma_wsd_nltk,jaccard_lemma_wsd_clean_spacy,jaccard_lemma_wsd_spacy,jaccard_wsd_cleaned
0,4.500,0.6538,0.6538,0.6923,0.6538,0.6667,0.6000,0.6667,0.5556,0.5556,0.5556,0.5556,0.6000
1,5.000,0.2143,0.2143,0.2857,0.0769,0.0000,0.0769,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,4.250,0.6087,0.6087,0.7273,0.6087,0.7273,0.5455,0.7273,0.8571,0.8571,0.8571,0.8571,0.7500
3,4.500,0.4545,0.4000,0.2500,0.4545,0.2500,0.4000,0.2500,0.2500,0.2500,0.2500,0.2500,0.2500
4,5.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,5.000,0.4500,0.4500,0.6364,0.4500,0.6364,0.4500,0.6364,0.6364,0.6364,0.6364,0.6364,0.6364
455,4.750,0.6429,0.7143,0.2500,0.6429,0.2500,0.6154,0.2500,0.2500,0.2500,0.2500,0.2500,0.2500
456,5.000,0.6000,0.6000,0.6667,0.6000,0.6667,0.5263,0.6667,0.6250,0.6250,0.6250,0.6250,0.6250
457,4.000,0.6818,0.6818,0.8000,0.6818,0.8000,0.6190,0.8000,0.8750,0.8750,0.8750,0.8750,0.7500


In [ ]:
results = {'Category':['NLTK', 'spaCy', 'NLTK Clean', 'spaCy Clean'],'Token':[], 'Lemma':[],'WSD-Lemma':[],'WSD-Data-Clean':[]}
results['Token'].append(pearsonr(dt['gs'], 1-dt['jaccard_token_nltk'])[0])
results['Token'].append(pearsonr(dt['gs'], 1-dt['jaccard_token_spacy'])[0])
results['Token'].append(pearsonr(dt['gs'], 1-dt['jaccard_clean'])[0])
results['Token'].append(pearsonr(dt['gs'], 1-dt['jaccard_clean'])[0])
results['Lemma'].append(pearsonr(dt['gs'], 1-dt['jaccard_lemma_nltk'])[0])
results['Lemma'].append(pearsonr(dt['gs'], 1-dt['jaccard_lemma_clean_nltk'])[0])
results['Lemma'].append(pearsonr(dt['gs'], 1-dt['jaccard_lemma_spacy'])[0])
results['Lemma'].append(pearsonr(dt['gs'], 1-dt['jaccard_lemma_clean_spacy'])[0])
results['WSD-Lemma'].append(pearsonr(dt['gs'], 1-dt['jaccard_lemma_wsd_nltk'])[0])
results['WSD-Lemma'].append(pearsonr(dt['gs'], 1-dt['jaccard_lemma_wsd_spacy'])[0])
results['WSD-Lemma'].append(pearsonr(dt['gs'], 1-dt['jaccard_lemma_wsd_clean_nltk'])[0])
results['WSD-Lemma'].append(pearsonr(dt['gs'], 1-dt['jaccard_lemma_wsd_clean_spacy'])[0])
results['WSD-Data-Clean'].append('NA')
results['WSD-Data-Clean'].append('NA')
results['WSD-Data-Clean'].append(pearsonr(dt['gs'], 1-dt['jaccard_wsd_cleaned'])[0])
results['WSD-Data-Clean'].append('NA')
results = pd.DataFrame(results)
results.set_index('Category')

,Token,Lemma,WSD-Lemma,WSD-Data-Clean
Category,,,,
NLTK,0.4505,0.4493,0.4458,NA
spaCy,0.4609,0.4810,0.4576,NA
NLTK Clean,0.4679,0.4768,0.4458,0.4416
spaCy Clean,0.4679,0.4912,0.4576,NA


# Conclusion

In this lab session excercise, we compared previous sentence pairs using Jaccard distance with and without lemmatization and data cleaning. And now with the Word Sense Desambiguation process, using both the "raw" lemmas and the lemmas with a previous data cleaning.

We can see in the table above that even if it is counter-intuitive the results of the pearson correlation between the previous lab (lemmas and words) outperformed the ones with the WSD both with words and lemmas, and with and without the data cleaning process.

Lesk, both when performed on the lemmatized set and the unlemmatized set, performed better than the document analysis from Lab 2. In both cases, this is likely because the context that the Lesk algorithm uses is supporting the evaluation of word similarities and creates a more accurate picture of the meaning of each word. Lesk occasionally performed better on the lemmatized set than on the unlemmatized set, with 24 of the instances having lower Jaccard coefficients for the lemmatized sets. Contrary to our expectations, the lemmatization does not seem to have a negative impact on the context. In other words, lemmatizing the sentence does not strip any meaning from the context. In any case, however, the improvement in performance when using Lesk is very small, despite our predictions that the improvement would be much greater, since we hypothesized that having context would improve performance.

Lesk, both when performed on the lemmatized set and the unlemmatized set, performed worse than just lemmatizing the words as done in Lab 3, as shown above. One possible reason for this may be that, when taking more words into account when determining similarity (i.e. by adding the context), there is more room for error. If the wrong synset is determined for some word in the context, then it will impact the interpreted meaning of the whole sentence. With just lemmatizing, on the other hand, as done in Lab 3, there is less chance of error, since only the structure of each word is being changed and examined, without the subjective interpretations of meaning used with context analysis.
